In [0]:
import pandas as pd
import numpy as np
import ast

In [0]:
from sklearn.utils import shuffle

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [0]:
data = pd.read_csv('/content/iMDB Films Datas - iMDB Films Datas.csv')
# Dataset link // https://www.kaggle.com/melikhanhosdogdu/imdb-movies-comprehensive-dataset

In [0]:
len(data)

9754

In [0]:
data.loc[data['imdbRaiting'] <= 5 , 'imdbRaiting'] = 0

In [0]:
data.loc[data['imdbRaiting'] >= 7 , 'imdbRaiting'] = 1

In [0]:
data.head()

,imdbRaiting,storyline
0,1.0,Chronicles the experiences of a formerly succe...
1,1.0,Set within a year after the events of Batman B...
2,1.0,"Dom Cobb is a skilled thief, the absolute best..."
3,1.0,A nameless first person narrator (Edward Norto...
4,1.0,Jules Winnfield (Samuel L. Jackson) and Vincen...


In [0]:
df_1 = data[data.imdbRaiting == 1]

In [0]:
len(df_1)

3342

In [0]:
df_0 = data[data.imdbRaiting == 0]

In [0]:
df_0

,imdbRaiting,storyline
505,0.0,"When Anastasia Steele, a literature student, g..."
600,0.0,After Bella recovers from the vampire attack t...
713,0.0,Batman and Robin are back working side-by-side...
751,0.0,Bella once again finds herself surrounded by d...
769,0.0,"Bella and Edward celebrate their wedding, but,..."
...,...,...
9674,0.0,Rip is the World Wrestling Federation champion...
9687,0.0,Danica is a successful worker at a marketing f...
9734,0.0,"Jason Crockett is an aging, grumpy, physically..."
9738,0.0,A young professional woman (Simpson) unwitting...


In [0]:
df = pd.concat([df_0, df_1], ignore_index= True)

In [0]:
df = shuffle(df)

In [0]:
storylineTRY = df["storyline"].values.tolist()
ortalama = list()
for story in storylineTRY:
  ortalama.append(len(story.split()))

ortalamaSayi = sum(ortalama) / len(ortalama) 
ortalamaSayi

95.47266881028939

In [0]:
df

,imdbRaiting,storyline
2159,1.0,Christian is the respected curator of a contem...
417,0.0,Set 15 years in the future in post-earthquake ...
4003,1.0,Ollie Dee and Stanley Dum try to borrow money ...
3195,1.0,Kermit the Frog and Fozzie Bear are newspaper ...
2794,1.0,Details the unconventional life of Dr. William...
...,...,...
2696,1.0,"A young woman, Pursy (played by Scarlett Johan..."
1001,1.0,Following the death of his employer and mentor...
2460,1.0,"Located in the US Rust Belt, Charlestown is ho..."
3796,1.0,The abuse Rocky Barbella endures at the hand o...


In [0]:
df.to_csv("iMDB_data_last_version", sep = "\t", index = False)

In [0]:
tokenizer = Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=" ",
    char_level=False,
    oov_token=None,
    document_count=0,
)

In [0]:
imdbRaiting = df["imdbRaiting"].values.tolist()

In [0]:
storyLine = df['storyline'].values.tolist()

In [0]:
tokenizer.fit_on_texts(storyLine)

In [0]:
print(str(dict(tokenizer.word_index.items())))

{'the': 1, 'a': 2, 'and': 3, 'to': 4, 'of': 5, 'in': 6, 'his': 7, 'is': 8, 'he': 9, 'with': 10, 'her': 11, 'for': 12, 'that': 13, 'as': 14, 'on': 15, 'who': 16, 'by': 17, 'an': 18, 'their': 19, 'they': 20, 'but': 21, 'has': 22, 'from': 23, 'him': 24, 'when': 25, 'she': 26, 'are': 27, 'at': 28, 'it': 29, 'life': 30, 'one': 31, 'be': 32, 'out': 33, 'after': 34, 'up': 35, 'this': 36, 'will': 37, 'into': 38, 'new': 39, 'all': 40, 'have': 41, 'not': 42, 'two': 43, 'them': 44, 'man': 45, 'young': 46, 'family': 47, 'only': 48, 'love': 49, 'which': 50, 'was': 51, 'about': 52, 'find': 53, 'while': 54, 'world': 55, 'time': 56, 'old': 57, 'get': 58, 'father': 59, 'years': 60, 'where': 61, 'can': 62, 'what': 63, 'story': 64, 'more': 65, 'being': 66, 'own': 67, 'finds': 68, 'other': 69, 'wife': 70, 'war': 71, 'so': 72, 'back': 73, 'must': 74, 'mother': 75, 'home': 76, 'friends': 77, 'been': 78, 'however': 79, 'year': 80, 'way': 81, 'help': 82, 'school': 83, 'now': 84, 'through': 85, 'there': 86, 'n

In [0]:
cutoff = int(len(storyLine) * 0.80)
imdbRaiting_train, imdbRaiting_test = imdbRaiting[:cutoff], imdbRaiting[cutoff:]
storyLine_train, storyLine_test = storyLine[:cutoff], storyLine[cutoff:]

In [0]:
print(len(imdbRaiting_train),len(imdbRaiting_test))

3234 809


In [0]:
with open("/content/tokenizer.txt","r") as txt_dosyasi:
  txt_tokenizer = txt_dosyasi.read()

In [0]:
txt_tokenizer = ast.literal_eval(txt_tokenizer)

In [0]:
def tokenlestir(listOfStoryLines):
    storyLines = []
    for s_storyLine in listOfStoryLines:
        storyLine = []
        for word in s_storyLine.split():
            if (len(storyLine) < 95 and word in txt_tokenizer):
                storyLine.append(txt_tokenizer[word])
                
        if (len(storyLine) < 95):
            zeros = list(np.zeros(95 - len(storyLine), dtype = int))
            storyLine = zeros + storyLine
            
        storyLines.append(storyLine)
    return np.array(storyLines, dtype = np.dtype(np.int32))

In [0]:
egitimKume = tokenlestir(storyLine_train)

In [0]:
egitimKume[50]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,   997,  2249,     5,     7,   404,
         250,    14,     2,     6,     7,  1004,  9491,   321,    35,
           4,     1,    56,     5,     7,  5953,    28,    50,     9,
        2146,     2,  4168,    23,     1,   515,    12,     7,   677,
       17355,     4,     1,    36,     8,     1,    64,     5,    16,
        3447,     3, 12253,     6,     7,    67,   997,   590,    12,
          50,     9,  5293,     7,   438], dtype=int32)

In [0]:
testKume = tokenlestir(storyLine_test)

In [0]:
len(txt_tokenizer)

29312

In [0]:
model = Sequential()

model.add(Embedding(input_dim=(len(txt_tokenizer) + 1),
                    output_dim=95,
                    input_length=95,
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 95, 95)            2784735   
_________________________________________________________________
gru_6 (GRU)                  (None, 95, 16)            5424      
_________________________________________________________________
gru_7 (GRU)                  (None, 95, 8)             624       
_________________________________________________________________
gru_8 (GRU)                  (None, 4)                 168       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 2,790,956
Trainable params: 2,790,956
Non-trainable params: 0
_________________________________________________________________


In [0]:
print(type(egitimKume), type(imdbRaiting_train))
egitimKume = egitimKume.tolist()

<class 'numpy.ndarray'> <class 'list'>


In [0]:
model.fit(egitimKume, imdbRaiting_train, epochs=5, batch_size=256)

Epoch 1/5
13/13 [==============================] - 2s 190ms/step - loss: 0.0464 - accuracy: 0.9978
Epoch 2/5
13/13 [==============================] - 2s 187ms/step - loss: 0.0447 - accuracy: 0.9978
Epoch 3/5
13/13 [==============================] - 2s 185ms/step - loss: 0.0432 - accuracy: 0.9978
Epoch 4/5
13/13 [==============================] - 2s 179ms/step - loss: 0.0418 - accuracy: 0.9978
Epoch 5/5
13/13 [==============================] - 2s 181ms/step - loss: 0.0405 - accuracy: 0.9978
